In [5]:
import os 
import json 
from pyssem.model import Model

with open('three_species.json') as f:
        simulation_data = json.load(f)

scenario_props = simulation_data["scenario_properties"]

# Create an instance of the pySSEM_model with the simulation parameters
model = Model(
    start_date=scenario_props["start_date"].split("T")[0],  # Assuming the date is in ISO format
    simulation_duration=scenario_props["simulation_duration"],
    steps=scenario_props["steps"],
    min_altitude=scenario_props["min_altitude"],
    max_altitude=scenario_props["max_altitude"],
    n_shells=scenario_props["n_shells"],
    launch_function=scenario_props["launch_function"],
    integrator=scenario_props["integrator"],
    density_model=scenario_props["density_model"],
    LC=scenario_props["LC"],
    v_imp = scenario_props.get("v_imp", None),
    fragment_spreading=scenario_props.get("fragment_spreading", True),
    parallel_processing=scenario_props.get("parallel_processing", False),
    baseline=scenario_props.get("baseline", False)
)

species = simulation_data["species"]

species_list = model.configure_species(species)

model.initial_population()
model.build_model()

times = [0, 1, 2, 3, 4, 5]

output = model.integrate(times)

Splitting species N into 2 species with masses [0.00141372, 0.567].
Added 1 active species, 3 debris species, and 1 rocket body species to the simulation.
Pairing the following active species to debris classes for PMD modeling...
['S']
Matched species S to debris species N_500kg.
    Name: N_0.00141372kg
    pmd_linked_species: []
    Name: N_0.567kg
    pmd_linked_species: []
    Name: N_500kg
    pmd_linked_species: ['S']
As no file is provided. Downloading a launch file...:
Initial URL Response Status Code: 200
Download URL Response Status Code: 200
Content-Length: 324434411


KeyboardInterrupt: 